In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
new_model="meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(new_model)
model = AutoModelForCausalLM.from_pretrained(
    new_model,
    load_in_4bit=True,
    device_map="auto",
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import torch.nn.functional as F
import torch
def log_prob(logits,ids):
    ## DPOのlog_Pi.ipynbで動作を確認

    """
        Args:
        logits (torch.Tensor): シーケンスの logits (batch_size, seq_len, vocab_size)
        ids (torch.Tensor): 選ばれたシーケンスのラベル IDs (batch_size, seq_len)
    """
    # Chosen の log probabilities を計算
    log_probs = F.log_softmax(logits, dim=-1)  # (batch_size, seq_len, vocab_size)
    chosen_selected_log_probs = torch.gather(
        log_probs, dim=-1, index=ids.unsqueeze(-1)  # (batch_size, seq_len, 1)
    ).squeeze(-1)  # (batch_size, seq_len)
    log_prob = chosen_selected_log_probs.sum(dim=-1)  # (batch_size,)
    return log_prob

In [ ]:
from datasets import load_dataset
ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")


In [ ]:
dataset = ds["train"]

In [ ]:
win_prob = 0
for i in range(len(dataset)):
    chosen_text=ds["train"][i]["prompt"]+ds["train"][i]["chosen"]
    rejected_text=ds["train"][i]["prompt"]+ds["train"][i]["rejected"]
    #WIN
    chosen_input_ids = tokenizer.encode(chosen_text,return_tensors="pt")
    chosen_dist_logits = model(chosen_input_ids).logits
    #LOSE
    rejected_input_ids = tokenizer.encode(rejected_text,return_tensors="pt")
    rejected_dist_logits = model(rejected_input_ids).logits
    #Logit
    chosen_prob=log_prob(chosen_dist_logits,chosen_input_ids)
    rejected_prob=log_prob(rejected_dist_logits,rejected_input_ids)
    if chosen_prob > rejected_prob:
        win_prob+=1


tensor([-1145.], dtype=torch.float16, grad_fn=<SumBackward1>)